In [1]:
! wget https://raw.githubusercontent.com/rewire-online/edos/main/data/edos_labelled_aggregated.csv
! pip install transformers
! pip install optuna
! pip install sentencepiece
! pip install gdown
! pip install sentence-transformers

! gdown "1UjEYJOauAanJ0UzuSRrCVLn6GHvJyWus&confirm=t"
! gdown --id 16YwJWsljY2r2R4Fzh4xLilSbPjowlZr_
! gdown --id 1YDr6ejvJPiQL3HAWR7i71GanzWsfxlQo
! gdown --id 11hYRWLmxyWd3j-rQsKTPS1fCuQvSFWcl
! gdown --id 1z2FvuIDZPUTKXJ0_Z6VBvFEzRteTLIa5
! gdown --id 1hEX89Ffv_lu8uIoOuX5trX0RqYFiI0Of
! gdown --id 1MD4pEghAgkpkxhr9Tyx_cEqGggki_XBu
! gdown --id 1zCW194SsKQ6U3sUEJ1eB7PiIKcOfdki0
! gdown --id 1yh77Xp7mwjjaYrPX6Y8OQchXSSqYIso-

! gdown "1-2pA6nIe74QSNzinswFKCxbcIMgu6m3T&confirm=t"

! yes A | unzip train_data.zip
! unzip eval_A.zip
! unzip eval_B.zip
! unzip eval_C.zip

--2023-02-23 13:56:17--  https://raw.githubusercontent.com/rewire-online/edos/main/data/edos_labelled_aggregated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3846925 (3.7M) [text/plain]
Saving to: ‘edos_labelled_aggregated.csv’

edos_labelled_aggre 100%[===================>]   3.67M  --.-KB/s    in 0.02s   

2023-02-23 13:56:18 (240 MB/s) - ‘edos_labelled_aggregated.csv’ saved [3846925/3846925]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 28.4 MB/s eta 0:00:00
Looking in indexes: htt

In [2]:
! wget https://raw.githubusercontent.com/rewire-online/edos/main/data/edos_labelled_aggregated.csv

--2023-02-23 13:58:38--  https://raw.githubusercontent.com/rewire-online/edos/main/data/edos_labelled_aggregated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3846925 (3.7M) [text/plain]
Saving to: ‘edos_labelled_aggregated.csv.1’

edos_labelled_aggre 100%[===================>]   3.67M  --.-KB/s    in 0.01s   

2023-02-23 13:58:38 (283 MB/s) - ‘edos_labelled_aggregated.csv.1’ saved [3846925/3846925]



In [3]:
import numpy as np
import torch 
import json
import pickle
import unicodedata
import re
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import transformers
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import BertModel, BertTokenizer, DebertaTokenizer, DebertaModel, RobertaTokenizer, RobertaModel, ElectraTokenizer, ElectraModel
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
# from torch_geometric.nn import GCNConv, GATConv, TransformerConv
import pandas as pd
import os
from collections import defaultdict, namedtuple, OrderedDict
from torch.utils.data import Dataset, DataLoader
from itertools import count 
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW, Adam, RMSprop
from copy import deepcopy
from sklearn.utils import shuffle
from typing import Union, Callable
import random
import gdown
from torch import Tensor
from typing import Optional, Tuple
import pickle as pk
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
# from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

seeds = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
seed_idx = 1
seed = seeds[seed_idx]
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

from transformers import AutoTokenizer, AutoModel
deberta = 'microsoft/deberta-v3-large' 
roberta = 'roberta-large'
model_name = deberta
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)


train_data = pd.read_csv('train_all_tasks.csv')
eval_data_A = pd.read_csv('dev_task_a_entries.csv')
eval_data_B = pd.read_csv('dev_task_b_entries.csv')
eval_data_C = pd.read_csv('dev_task_c_entries.csv')
all_data_edos = pd.read_csv('edos_labelled_aggregated.csv')


def tolist(tensor):
  return tensor.detach().cpu().tolist()

def map_names_2_ids(names):
  A = dict()
  B = dict()
  for id, name in enumerate(names):
    A[name] = id
    B[id] = name
  return A, B

def dist(x1, x2):
  return (x1 - x2).pow(2).sum(-1).sqrt()

def entropy(logits):
  probs = F.softmax(logits, dim=-1)
  ent = -torch.sum((probs * torch.log2(probs)),dim=1)
  return ent

train_data = train_data[train_data['label_sexist'] == 'sexist'].reset_index(drop=True)
test_data = deepcopy(all_data_edos[(all_data_edos['split'] == 'test') & (all_data_edos['label_sexist'] =='sexist')]).reset_index(drop=True)

label_category_raw = np.unique(train_data['label_category']).tolist()
label_category_map, category_label_map = map_names_2_ids(label_category_raw)
train_data['Tag_B'] = [label_category_map[i[1]] for i in train_data['label_category'].iteritems()]
label_category = list(label_category_map.keys())
label_category = list(map(lambda x: re.sub('^\d+\.\d*', '', x).strip(), label_category))
eval_label_B = pd.read_csv('dev_task_b_labels.csv')
eval_B = eval_data_B.merge(eval_label_B, on='rewire_id')
eval_B['Tag_B'] = [label_category_map[i[1]] for i in eval_B['label'].iteritems()]
test_data['Tag_B'] = [label_category_map[i[1]] for i in test_data['label_category'].iteritems()]

num_labels_B = len(train_data['label_category'].unique())

label_vector_raw = np.unique(train_data['label_vector']).tolist()
label_vector_map, vector_label_map = map_names_2_ids(label_vector_raw)
train_data['Tag_C'] = [label_vector_map[i[1]] for i in train_data['label_vector'].iteritems()]
label_vector = list(label_vector_map.keys())
label_vector = list(map(lambda x: re.sub('^\d+\.\d*', '', x).strip(), label_vector))
eval_label_C = pd.read_csv('dev_task_c_labels.csv')
eval_C = eval_data_C.merge(eval_label_C, on='rewire_id')
eval_C['Tag_C'] = [label_vector_map[i[1]] for i in eval_C['label'].iteritems()] 
num_labels_C = len(train_data['label_vector'].unique())

train_dataframe = train_data
eval_dataframe = eval_B
test_dataframe = test_data
new_l = label_category

class_weights = compute_class_weight(class_weight='balanced', classes=np.array(list(range(num_labels_B))), y=train_data['Tag_B'].values.tolist()).tolist()

class SexistDataset(Dataset):

  def __init__(self, dataframe, tokenizer, max_length=100, is_test=False):
    self.dataframe = dataframe
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.labels_names = f'{tokenizer.sep_token}'.join(new_l)
    self.is_test = is_test

    self.labels_tokens = []
    for label_name in new_l:
      label_tokens = tokenizer(label_name, add_special_tokens=False)
      self.labels_tokens.append(label_tokens['input_ids'])

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, idx):
    sample = self.dataframe.loc[idx]
    tokenized_text = tokenizer(
          sample['text'],
          max_length=self.max_length,
          padding='max_length',
          truncation='only_first',
          return_tensors='pt')

    # find the first token of labels
    input_ids = tokenized_text['input_ids']
    labels_start = (input_ids == tokenizer.sep_token_id).nonzero().contiguous().view(-1).tolist()[1] + 2

    labels_tokens_span = []
    c_token = labels_start
    # print(labels_start)
    for label_tokens in self.labels_tokens:

      labels_tokens_span.append([c_token, c_token + len(label_tokens) - 1])
      c_token += len(label_tokens) + 1
    tokenized_text['labels_tokens_span'] = torch.tensor(labels_tokens_span)
    if not self.is_test:
      labels_B = torch.LongTensor([sample['Tag_B']])
      tokenized_text['Tag_B'] = labels_B
    return tokenized_text


class UnlabeledDataset:
  def __init__(self, dataframe, tokenizer, max_length=70, batch_size=10):
    self.data = dataframe
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.batch_size = batch_size
    self.idxs = np.random.permutation(range(len(self.data)))
    self.current_idx = 0

  def __len__(self):
    return len(self.dataframe)

  def combine_tenors(self, tensors_list):
    combined_tensor = dict()
    keys = tensors_list[0].keys()
    for key in keys:
      combined_tensor[key] = torch.cat([tensor[key] for tensor in tensors_list])
    return combined_tensor

  def next(self):
    if self.current_idx >= len(self.data) - 1:
      self.current_idx = 0
    compact_data = self.data.iloc[self.idxs[self.current_idx: self.current_idx + self.batch_size]]
    self.current_idx += self.batch_size
    all_t = [self.tokenize(data) for _, data in compact_data.iterrows()]
    T = self.combine_tenors(all_t)
    return T

  def tokenize(self, data):
    return self.tokenizer(data['text'], padding='max_length', max_length=self.max_length, truncation=True, return_tensors='pt')


class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim: int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int = 512, num_heads: int = 8):
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

    def forward(
            self,
            query: Tensor,
            key: Tensor,
            value: Tensor,
            mask: Optional[Tensor] = None
    ) -> Tuple[Tensor, Tensor]:
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)      # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1)  # BxNxQ_LENxK_LEN

        context, attn = self.scaled_dot_attn(query, key, value, mask)

        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn

def exists(value):
    return value is not None


def default(value, default):
    if exists(value):
        return value
    return default


def inf_norm(x):
    return torch.norm(x, p=float("inf"), dim=-1, keepdim=True)


def kl_loss(input, target, reduction="batchmean"):
    return F.kl_div(
        F.log_softmax(input, dim=-1),
        F.softmax(target, dim=-1),
        reduction=reduction,
    )


def sym_kl_loss(input, target, reduction="batchmean", alpha=1.0):
    return alpha * F.kl_div(
        F.log_softmax(input, dim=-1),
        F.softmax(target.detach(), dim=-1),
        reduction=reduction,
    ) + F.kl_div(
        F.log_softmax(target, dim=-1),
        F.softmax(input.detach(), dim=-1),
        reduction=reduction,
    )


def js_loss(input, target, reduction="batchmean", alpha=1.0):
    mean_proba = 0.5 * (
        F.softmax(input.detach(), dim=-1) + F.softmax(target.detach(), dim=-1)
    )
    return alpha * (
        F.kl_div(F.log_softmax(input, dim=-1), mean_proba, reduction=reduction)
        + F.kl_div(F.log_softmax(target, dim=-1), mean_proba, reduction=reduction)
    )


def ntxent(logits, labels, temp=.07):
  def ntx_loss(a, p, n, temp=temp):
    a = a.unsqueeze(0) if a.dim() == 1 else a
    p = p.unsqueeze(0) if p.dim() == 1 else p
    n = n.unsqueeze(0) if n.dim() == 1 else n
    assert a.dim() == 2
    assert p.dim() == 2
    assert n.dim() == 2
    a_p = a
    a_n = a.repeat(n.shape[0], 1)
    p_sim = F.cosine_similarity(a_p, p, dim=-1) / temp
    n_sim = F.cosine_similarity(a_n, n, dim=-1) / temp

    # apply numeric stability
    max_val = torch.max(n_sim).detach()
    numerator = torch.exp(p_sim - max_val)
    denominator = torch.exp(n_sim - max_val).sum()
    loss = -torch.log(numerator / (denominator + numerator) + 1e-6)
    if loss.isnan():
      print(numerator, denominator)
      print(p_sim)
      print(len(n))
    # print(loss)
    return loss.mean()

  def dist(x1, x2):
    return (x1 - x2).pow(2).sum(-1).sqrt()

  con_losses = list()
  for i, (logit, label) in enumerate(zip(logits, labels)):
    ps = (labels == label)
    ns = (labels != label).nonzero().view(-1)
    ps[i] = False
    ps = ps.nonzero().view(-1)
    if len(ns):
      for p in ps:
        a_logit = logits[i]
        p_logit = logits[p]
        ns_logit = logits[ns]
        A = ntx_loss(a_logit, p_logit, ns_logit)
        con_losses.append(A)

  if len(con_losses) > 0:
    all_con_loss = torch.stack(con_losses).mean()
  else:
    all_con_loss = torch.tensor(0.)
  return all_con_loss


class SMARTLoss(nn.Module):
    
    def __init__(
        self,
        model: nn.Module,
        loss_fn: Callable,
        loss_last_fn: Callable = None, 
        norm_fn: Callable = inf_norm, 
        num_steps: int = 1,
        step_size: float = 1e-3, 
        epsilon: float = 1e-6,
        noise_var: float = 1e-5
    ) -> None:
        super().__init__()
        self.model = model 
        self.loss_fn = loss_fn
        self.loss_last_fn = default(loss_last_fn, loss_fn)
        self.norm_fn = norm_fn
        self.num_steps = num_steps 
        self.step_size = step_size
        self.epsilon = epsilon 
        self.noise_var = noise_var
     
    @torch.enable_grad()   
    def forward(self, embed, state, batch_labels_tokens_span, attention_mask):
        noise = torch.randn_like(embed, requires_grad = True) * self.noise_var 
        
        # Indefinite loop with counter 
        for i in count():
            # Compute perturbed embed and states 
            embed_perturbed = embed + noise 
            state_perturbed, _ = self.model(embed_perturbed, batch_labels_tokens_span, True, attention_mask) 
            # Return final loss if last step (undetached state)
            if i == self.num_steps: 
                return self.loss_last_fn(state_perturbed, state) 
            # Compute perturbation loss (detached state)
            loss = self.loss_fn(state_perturbed, state.detach())
            # Compute noise gradient ∂loss/∂noise
            noise_gradient, = torch.autograd.grad(loss, noise)
            # Move noise towards gradient to change state as much as possible 
            step = noise + self.step_size * noise_gradient 
            # Normalize new noise step into norm induced ball 
            step_norm = self.norm_fn(step)
            noise = step / (step_norm + self.epsilon)
            # Reset noise gradients for next step
            noise = noise.detach().requires_grad_()

class PGD():

    def __init__(self, model,emb_name,epsilon=1.,alpha=0.3):
        # The emb_name parameter should be replaced with the parameter name of the embedding in your model
        self.model = model
        self.emb_name = emb_name
        self.epsilon = epsilon
        self.alpha = alpha
        self.emb_backup = {}
        self.grad_backup = {}

    # adversarial training : attack to change embedding abit with regards projected gradiant descent
    def attack(self,first_strike=False):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                if first_strike:
                    # print('tt', param.data)
                    self.emb_backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    # Compute new params
                    r_at = self.alpha * param.grad / norm
                    param.data.add_(r_at)
                    param.data = self.project(name, param.data, self.epsilon)

    # Restore to the back-up embeddings
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and self.emb_name in name:
                assert name in self.emb_backup
                param.data = self.emb_backup[name]
        self.emb_backup = {}

    # Project Gradiant Descent
    def project(self, param_name, param_data, epsilon):
        r = param_data - self.emb_backup[param_name]
        if torch.norm(r) > epsilon:
            r = epsilon * r / torch.norm(r)
        return self.emb_backup[param_name] + r

    # Back-up parameters
    def backup_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and 'pooler' not in name:
                self.grad_backup[name] = param.grad.clone()

    # Restore grad parameters
    def restore_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and 'pooler' not in name:
                param.grad = self.grad_backup[name]





class SexistModel(nn.Module):

    def __init__(self, model):
        super().__init__()
        self.transformer = model
        hidden_size = self.transformer.config.hidden_size
        self.dropout = nn.Dropout(p=.3)
        self.head = nn.Linear(hidden_size, num_labels_B)

    def integrate(self, batch_output, batch_labels_tokens_span):
      batch_size = batch_output.shape[0]
      integrated_batch = []
      for i in range(batch_size):
        integrated_sample_labels = []
        output = batch_output[i]
        labels_tokens_span = batch_labels_tokens_span[i]
        for label_tokens_span in labels_tokens_span:
          integrated_label = output[label_tokens_span[0].item(): label_tokens_span[1].item() + 1].mean(0).view(-1)
          assert integrated_label.shape[0] == self.transformer.config.hidden_size
          integrated_sample_labels.append(integrated_label)
        integrated_sample_labels = torch.stack(integrated_sample_labels)
        integrated_batch.append(integrated_sample_labels)
      integrated_batch = torch.stack(integrated_batch)
      return integrated_batch

    def forward(self, x, batch_labels_tokens_span, vat=False, attention_mask=None):
        if vat:
          hidden = self.transformer(inputs_embeds=x, attention_mask=attention_mask).last_hidden_state
        else:
          hidden = self.transformer(**x).last_hidden_state
        cls = hidden[:, 0, :]
        x = self.head(cls)
        x = x.view(-1, num_labels_B)
        return x, hidden[:, 0, :]




def train(dataloader, model, device, loss_fn, optimizer, scheduler, stage, ul_dataset, use_contrastive=False,
          use_adv=True, use_vadv=False, use_ul=False, vat_weight=.5, ul_weight=.5, con_weight=.5, adv_use_every_layer=True):
  
  model.train()
  named_weights = [n for n, _ in model.named_parameters() if 'dense.weight' in n and 'pooler' not in n] + ["word_embeddings."]
  loss_collection = [[], [], [], [], []]
  for step, data in enumerate(dataloader):

    if adv_use_every_layer:
      rand_layer = random.sample(named_weights, 1)[0] 
      adv_layer = rand_layer
    else:
      adv_rand = random.uniform(0, 1) 
      if adv_rand > .7:
        adv_layer = "word_embeddings."
      else:
        rand_layer = random.sample(named_weights, 1)[0] 
        adv_layer = rand_layer
    pgd = PGD(
      model=model,
      emb_name=adv_layer
    )


    c_batch_size = data['input_ids'].shape[0]
    labels = data.pop('Tag_B').to(device).view(-1)
    for key in data:
      data[key] = data[key].to(device).view(c_batch_size, -1)
    batch_labels_tokens_span = data.pop('labels_tokens_span').view(-1, num_labels_B, 2)

    logits, _ = model(data, batch_labels_tokens_span)

    ce_loss = loss_fn(logits, labels)
    ce_loss.backward()
    loss_collection[0].append(ce_loss.item())


    
    if use_adv:
      # PGD Start
        pgd.backup_grad()
        attack_times = 2
        for attack_time in range(attack_times):
            # Add adversarial perturbation to the embedding, backup param.data during the first attack
            pgd.attack(first_strike=(attack_time==0))
            if attack_time != attack_times-1:
              model.zero_grad()
            else:
              pgd.restore_grad()
            logits_adv, _ = model(data, batch_labels_tokens_span)
            loss_adv = loss_fn(logits_adv, labels)
            loss_collection[1].append(loss_adv.item())
            loss_adv.backward()
        # Restore embedding parameters
        pgd.restore() 

    if use_contrastive:
      embeddings = model.get_embeddings(data['input_ids'].to(device))
      model.set_attention_mask(data['attention_mask'].to(device))
      logits = model(embeddings, batch_labels_tokens_span)
      con_loss = ntxent(logits, labels) * con_weight
      if con_loss.requires_grad:
        con_loss.backward()
      loss_collection[2].append(con_loss.item())


    if use_vadv:
      vat_loss_fn = SMARTLoss(model = model, loss_fn = kl_loss, loss_last_fn = sym_kl_loss)
      # Compute VAT loss
      embeddings = model.transformer.embeddings(data['input_ids'], data['token_type_ids'])
      logits, _ = model(embeddings, batch_labels_tokens_span, True, data['attention_mask'])
      vat_loss = vat_loss_fn(embeddings, logits, batch_labels_tokens_span, data['attention_mask']) 
      # Merge losses 
      vat_loss = vat_weight * vat_loss
      vat_loss.backward()
      loss_collection[3].append(vat_loss.item())    


    if use_ul:
      ul_data = ul_dataset.next()
      c_batch_size = ul_data['input_ids'].shape[0]
      for key in ul_data:
        ul_data[key] = ul_data[key].to(device).view(c_batch_size, -1)

      ul_embeddings = model.get_embeddings(ul_data['input_ids'].to(device))
      model.set_attention_mask(ul_data['attention_mask'].to(device))
      ul_logits = model(ul_embeddings)

      vat_loss_fn = SMARTLoss(model = model, loss_fn = kl_loss, loss_last_fn = sym_kl_loss)
      vat_loss = vat_loss_fn(ul_embeddings, ul_logits) 
      ul_loss = ul_weight * vat_loss
      loss_collection[4].append(ul_loss.item())
      ul_loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()
    
    if len(loss_collection[0]) % log_step == 0:
      print(f'EPOCH [{epoch + 1}/{epochs}] | STEP [{step + 1}/{len(train_dataloader)}] | CE Loss {round(sum(loss_collection[0]) / (len(loss_collection[0]) + 1e-8), 4)}')
      print(f'EPOCH [{epoch + 1}/{epochs}] | STEP [{step + 1}/{len(train_dataloader)}] | ADV Loss {round(sum(loss_collection[1]) / (len(loss_collection[1]) + 1e-8), 4)}')
      print(f'EPOCH [{epoch + 1}/{epochs}] | STEP [{step + 1}/{len(train_dataloader)}] | CON Loss {round(sum(loss_collection[2]) / (len(loss_collection[2]) + 1e-8), 4)}')
      print(f'EPOCH [{epoch + 1}/{epochs}] | STEP [{step + 1}/{len(train_dataloader)}] | VAT Loss {round(sum(loss_collection[3]) / (len(loss_collection[3]) + 1e-8), 4)}')
      print(f'EPOCH [{epoch + 1}/{epochs}] | STEP [{step + 1}/{len(train_dataloader)}] | UL Loss {round(sum(loss_collection[4]) / (len(loss_collection[4]) + 1e-8), 4)}')
      print('------------------------------------------------')
      loss_collection = [[] for _ in range(5)]


def eval(dataloader, model, device):
  with torch.no_grad():
    model.eval()
    all_preds = list()
    all_hidden = list()
    for data in dataloader:
      c_batch_size = data['input_ids'].shape[0]
      for key in data:
        data[key] = data[key].to(device).view(c_batch_size, -1)
      batch_labels_tokens_span = data.pop('labels_tokens_span').view(-1, num_labels_B, 2)
      Tag_B = data.pop('Tag_B').to(device).view(-1)
      logits, hiddens = model(data, batch_labels_tokens_span)
      all_hidden.extend(tolist(hiddens))
      preds = tolist(logits.argmax(1).view(-1))
      all_preds.extend(preds)
  return all_preds, all_hidden


def test(dataloader, model, device):
  with torch.no_grad():
    model.eval()
    all_preds = list()

    for data in dataloader:
      c_batch_size = data['input_ids'].shape[0]
      for key in data:
        data[key] = data[key].to(device).view(c_batch_size, -1)
      batch_labels_tokens_span = data.pop('labels_tokens_span').view(-1, num_labels_B, 2)
      logits, _ = model(data, batch_labels_tokens_span)
      preds = tolist(logits.argmax(1).view(-1))
      all_preds.extend(preds)
  return all_preds

epochs = 30
lr = 1e-5
beta_1 = .9
beta_2 = .999
eps = 1e-6
log_step = 100
batch_size = 10
weight_decay = 9e-3
max_length = 70
loss_file = 'loss.txt'
eval_file = 'eval.txt'

vat_weight = .5
ul_weight = .5
ent_weight = .5


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# sexist_model = ExtractedRoBERTa(deepcopy(model)).to(device)
sexist_model = SexistModel(deepcopy(model)).to(device)
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device)).to(device)
loss_collection = []

train_dataset = SexistDataset(train_dataframe, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

eval_dataset = SexistDataset(eval_dataframe, tokenizer, max_length)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

test_dataset = SexistDataset(test_dataframe, tokenizer, max_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

opt_step = 0
optimization_steps = epochs * len(train_dataloader)
warmup_ratio = .0
warmup_steps = int(optimization_steps * warmup_ratio)


optimizer = AdamW(sexist_model.parameters(), lr=lr, betas=(beta_1,beta_2), eps=eps, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps, 
    num_training_steps=optimization_steps)

best_f1 = 0.
best_model = None
transformers.logging.set_verbosity_error()



checkpoint_dir = '.'
filename = os.path.join(checkpoint_dir, 'best_ch.pt')


def save_model(epoch, model, optimizer, scheduler):
  filename = os.path.join(checkpoint_dir, 'best_ch.pt')
  torch.save(
      {'epoch': epoch,
       'model_state_dict': model.state_dict(),
       'optimizer_state_dict': optimizer.state_dict(),
       'scheduler_state_dict': scheduler.state_dict()}, 
        filename)

def load_model():
  if os.path.exists(filename):
    saved_dict = torch.load(filename)
    return True, saved_dict
  else:
    return False, None
    

def early_stop(scores, current_score, patience):
  if len(scores) < patience:
    return False
  else:
    for score in scores[-patience: ]:
      if current_score > score:
        return False
    return True

all_f1 = list()
patience = 2

# for epoch in range(epochs):
#   train(train_dataloader, sexist_model, device, loss_fn, optimizer, scheduler, 1, None)
#   preds_B_eval = eval(eval_dataloader, sexist_model, device)
#   f1_macro_B_eval = f1_score(eval_dataframe['Tag_B'].values.tolist(), preds_B_eval, average='macro')
#   all_f1.append(f1_macro_B_eval)
#   if f1_macro_B_eval > best_f1:
#     best_f1 = f1_macro_B_eval
#     best_preds = preds_B_eval
#     save_model(epoch + 1, sexist_model, optimizer, scheduler)

#   print(f'EPOCH [{epoch + 1}/{epochs}] | Current F1-Macro {round(f1_macro_B_eval * 100, 2)}')
#   print(f'EPOCH [{epoch + 1}/{epochs}] | Best F1-Macro {round(best_f1 * 100, 2)}')
#   print(confusion_matrix(eval_dataframe['Tag_B'].values.tolist(), preds_B_eval))

#   if early_stop(all_f1, f1_macro_B_eval, patience):
#     break
#   else:
#     print('not early stopping')


_, saved_dict = load_model()
sexist_model.load_state_dict(saved_dict['model_state_dict'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [4]:
def eval(dataloader, model, device):
  with torch.no_grad():
    model.eval()
    all_preds = list()
    all_hidden = list()
    for data in dataloader:
      c_batch_size = data['input_ids'].shape[0]
      for key in data:
        data[key] = data[key].to(device).view(c_batch_size, -1)
      batch_labels_tokens_span = data.pop('labels_tokens_span').view(-1, num_labels_B, 2)
      Tag_B = data.pop('Tag_B').to(device).view(-1)
      logits, hiddens = model(data, batch_labels_tokens_span)
      all_hidden.extend(tolist(hiddens))
      preds = tolist(logits.argmax(1).view(-1))
      all_preds.extend(preds)
  return all_preds, all_hidden

In [5]:
  preds_B_test, _ = eval(test_dataloader, sexist_model, device)
  f1_macro_B_test = f1_score(test_data['Tag_B'].values.tolist(), preds_B_test, average='macro')

In [6]:
preds_B_test, _ = eval(test_dataloader, sexist_model, device)
f1_macro_B_test = f1_score(test_data['Tag_B'].values.tolist(), preds_B_test, average='macro')
pr_B_test = precision_score(test_data['Tag_B'].values.tolist(), preds_B_test, average='macro')
rc_B_test = recall_score(test_data['Tag_B'].values.tolist(), preds_B_test, average='macro')

In [7]:
mat = confusion_matrix(test_data['Tag_B'].values.tolist(), preds_B_test)

In [8]:
f1_macro_B_test, pr_B_test, rc_B_test

(0.6778446509164594, 0.6916144996583754, 0.6775330287366629)